# EDA
This notebook contains EDA code and result for MIMIC-III dataset.  

### Notes
1. explore TEXT in noteevent with metadata analysis and visually scanning through records  
2. explore diagnosis distributions
3. explore diagnosis x TEXT to identify patterns

## TEXT

In [2]:
%%time
# load data
import sqlite3
import pandas as pd

# here we join with diagnoses_icd because we're only interested in charts with diagnoses

conn = sqlite3.connect('../database/mimic.db')
sql = 'SELECT DISTINCT a.hadm_id, a.category, a.description, a.text \
       FROM noteevents a \
       INNER JOIN diagnoses_icd b \
           ON a.hadm_id = b.hadm_id;'

df = pd.read_sql_query(sql, conn)

CPU times: user 3min 7s, sys: 49.5 s, total: 3min 56s
Wall time: 5min 58s


In [3]:
%%time
# load diagnoses with description
sql = 'SELECT DISTINCT a.hadm_id, a.icd9_code, b.short_title, b.long_title \
       FROM diagnoses_icd a \
       INNER JOIN d_icd_diagnoses b \
           ON a.icd9_code = b.icd9_code'

df_diag = pd.read_sql_query(sql, conn)

CPU times: user 2.53 s, sys: 482 ms, total: 3.01 s
Wall time: 3.11 s


In [4]:
# rename to lowercase for easier typing
df = df.rename(columns={'CATEGORY':'category',
                        'DESCRIPTION':'description',
                        'TEXT':'text'})

# remove white space in category and description
df['category'] = df['category'].str.strip()
df['description'] = df['description'].str.strip()

display(df.head())
print(df.shape)

,HADM_ID,category,description,text
0,174680,Discharge summary,Report,Admission Date: [**2147-11-17**] ...
1,117806,Discharge summary,Report,Admission Date: [**2154-4-30**] ...
2,184534,Discharge summary,Report,Admission Date: [**2172-3-26**] ...
3,195146,Discharge summary,Report,Admission Date: [**2198-11-23**] ...
4,154895,Discharge summary,Report,Admission Date: [**2190-5-16**] Discharge...


(1816038, 4)


In [5]:
df['category'].value_counts()

Nursing/other        815274
Radiology            378920
Nursing              199720
ECG                  137921
Physician            132833
Discharge summary     59568
Echo                  34037
Respiratory           31421
Nutrition              9266
General                8025
Rehab Services         5378
Social Work            2530
Case Management         950
Pharmacy                 99
Consult                  96
Name: category, dtype: int64

In [6]:
df['category'].value_counts(normalize=True)

Nursing/other        0.448930
Radiology            0.208652
Nursing              0.109976
ECG                  0.075946
Physician            0.073144
Discharge summary    0.032801
Echo                 0.018742
Respiratory          0.017302
Nutrition            0.005102
General              0.004419
Rehab Services       0.002961
Social Work          0.001393
Case Management      0.000523
Pharmacy             0.000055
Consult              0.000053
Name: category, dtype: float64

In [7]:
n_top = 10
freq_desc = df['description'].value_counts()[:n_top]
freq_desc

Report                               1042334
Nursing Progress Note                 170865
CHEST (PORTABLE AP)                   156240
Physician Resident Progress Note       59837
Respiratory Care Shift Note            30792
Nursing Transfer Note                  27973
CT HEAD W/O CONTRAST                   26891
Intensivist Note                       23769
CHEST (PA & LAT)                       21577
Physician Attending Progress Note      19429
Name: description, dtype: int64

In [8]:
freq_desc.sum()/df.shape[0]

0.8698645072404873

### Notes
Here we see different types of categories for each charts. We may be interested in seeing how the texts are different among each type of categories.  
  
__Report__ accounts for about 50% of all chart descriptions with top 10 most frequent description types representing about 87% of all charts.

### Action Item
1. for bigger categories, explore description breakout
2. explore text differences within each category and description
    * also scan for diagnosis to see if patterns are observed
3. from diagnosis, review charts to see if patterns can be observed

In [9]:
list_category = ['Nursing/other', 'Radiology', 'Nursing', 'ECG', 'Physician']
n_top = 5

for category in list_category:
    print(f'{category}: ')
    display(df[df['category'] == category]['description'].value_counts()[:n_top])
    print('\n')

Nursing/other: 


Report    815274
Name: description, dtype: int64



Radiology: 


CHEST (PORTABLE AP)           156240
CT HEAD W/O CONTRAST           26891
CHEST (PA & LAT)               21577
CHEST PORT. LINE PLACEMENT     19281
PORTABLE ABDOMEN                7452
Name: description, dtype: int64



Nursing: 


Nursing Progress Note    170861
Nursing Transfer Note     27972
Generic Note                571
Nursing 1900-0700           162
ICU Event Note               24
Name: description, dtype: int64



ECG: 


Report    137921
Name: description, dtype: int64



Physician: 


Physician Resident Progress Note     59821
Intensivist Note                     23749
Physician Attending Progress Note    19429
Physician Resident Admission Note     9659
ICU Note - CVI                        4393
Name: description, dtype: int64

In [270]:
# create subset of data to inspect
insp_category = 'Physician'
insp_descript = 'Physician Resident Progress Note'

df_inspect = df[(df['category'] == insp_category) & \
                (df['description'] == insp_descript)]\
                .sort_values('HADM_ID') \
                .reset_index(drop=True)

In [282]:
# from index, show how many notes for one hadm_id
index = 50
hadm_id = df_inspect.at[index, 'HADM_ID']
num_record = df[df['HADM_ID'] == hadm_id].shape[0]
print(f'HADM ID: {hadm_id} has {num_record} records.')

HADM ID: 100182 has 42 records.


In [283]:
# get diagnosese for the hadm_id
display(df_diag[df_diag['HADM_ID'] == hadm_id].sort_values('ICD9_CODE'))

,HADM_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
112878,100182,25000,DMII wo cmp nt st uncntr,Diabetes mellitus without mention of complicat...
112880,100182,2749,Gout NOS,"Gout, unspecified"
112869,100182,2761,Hyposmolality,Hyposmolality and/or hyponatremia
112881,100182,28521,Anemia in chr kidney dis,Anemia in chronic kidney disease
112883,100182,2989,Psychosis NOS,Unspecified psychosis
112890,100182,3669,Cataract NOS,Unspecified cataract
112874,100182,40390,Hy kid NOS w cr kid I-IV,"Hypertensive chronic kidney disease, unspecifi..."
112877,100182,412,Old myocardial infarct,Old myocardial infarction
112870,100182,4139,Angina pectoris NEC/NOS,Other and unspecified angina pectoris
112875,100182,41401,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...


In [284]:
print(df_inspect.at[index, 'text'])

Chief Complaint: transfer to CCU for respiratory distress
   24 Hour Events:
   [**2165-5-6**]:
   -3 a.m.: had LUQ pain. Received morphine 0.5 mg PO
   -5:30 a.m., went into Afib with normal HR and BP and no other EKG
   changes
   -spoke with Dr. [**Last Name (STitle) 2759**]: stopped Lasix gtt, free water restrict to 1500
   cc daily
   -Gave Isordil 10 mg ONCE PO tonight (not given Imdur or felodipine this
   a.m. given that she had twice normal carvedilol) just to cover CAD
   until a.m.
   - Stopped NAC given no renal artery stent for now.
   - Na corrected by 6 over 12 hours, K 3.7 but Cr 4.2 (didn't replete)
   Allergies:
   Atorvastatin
   muscle/bone pai
   Tylenol (Oral) (Dm Hb/Pseudoephed/Acetamin/Cp)
   muscle pain;
   Ibuprofen
   muscle/bone pai
   Rosuvastatin
   Abdominal pain;
   Last dose of Antibiotics:
   Infusions:
   Other ICU medications:
   Other medications:
   Changes to medical and family history:
   Review of systems is unchanged from admission except as no

In [285]:
hadm_id_all = df_inspect['HADM_ID'].drop_duplicates().tolist()
df_diag[df_diag['HADM_ID'].isin(hadm_id_all)]['LONG_TITLE'].value_counts()[:10]

Unspecified essential hypertension                                                                            1710
Congestive heart failure, unspecified                                                                         1590
Acute kidney failure, unspecified                                                                             1564
Atrial fibrillation                                                                                           1308
Acute respiratory failure                                                                                     1197
Other and unspecified hyperlipidemia                                                                          1175
Coronary atherosclerosis of native coronary artery                                                             989
Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled     934
Urinary tract infection, site not specified                                     

## Text Inspection Notes
Same hospital stays can result in multiple notes being generated. This causes an issue becuase diagnosis cannot be mapped to text with 1-to-1 relationship.  
From model's perspective, it may make sense to __concatenate text by HADM_ID__ so the labels (diagnosis) can be applied properly.
  
Check __prevelance of diagnosis__ within each category. This is a proxy becasue same hadm_id can have notes in different categories.

## Summary of each section
### Nursing/other
Progress notes on patient, with description of patient mental/physical/social conditions, medicine given, and other relevent notes.  
  
Dx - blood, heart, cardiac arrest, acute respiratory failure.  
Prevalence - heart, hepatitis vaccine, infection, kidney failure, respiratory failure  

### Radiology/CHEST (PORTABLE AP)
Usually has a _reason for examination_, technique used, brief description of patient condition, and findings.  
  
Dx - fracture, acute respiratory failure.  
Prevalence - hypertension, heart failure, kidney failure, diabetes, lipidemia, UTI

### Radiology/CT HEAD W/O CONTRAST
Text description tends to associate with brain injury or suspected conditions.  
  
Dx - fall, common health conditions, head injury (concussion), hemorrhage in brain  
Prevalence - hyptertension, respiratory/heart/kidney failure, diabetes, pneumonitis from food

### Radiology/CHEST (PA & LAT)
Dx - pulmonary collapse  
Prevalence - hyptertension, coronary, heart/kidney failure, diabetes, UTI

### Radiology/CHEST PORT. LINE PLACEMENT
Prevalence - about the same

### Radiology/PORTABLE ABDOMEN
Prevalence - sepsis

### ECG
small paragraph describing graph pattern

### Physicians
more listing style

# Conclusion

Upon investigaion, we have some idea of the chart format, the type of diagnoses associated with different chart types, as well as data type.  
  
__The next step__ will be concatenating all the text by _HADM_ID_ and creating training dataset.